In [11]:
import time
from urllib import FancyURLopener  # This is library that helps us create the headless browser
import urllib2
from random import choice #This library helps pick a random item from a list
import bs4 as bs
import datetime
from datetime import timedelta, date
import pandas as pd
from selenium import webdriver
import sys
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
import collections

In [12]:
user_agents = [
    'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
    'Mozilla/5.0 (Macintosh; Intel MaPc OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
]

class MyOpener(FancyURLopener, object):
    version = choice(user_agents)

def getDate(i):
    return pd.to_datetime(date,infer_datetime_format=True)

def getCompanyNames(link):
    Companies = []
    myopener = MyOpener()
    page=myopener.open('https://www.sec.gov'+link)
    
    html = page.read()
    soup = bs.BeautifulSoup(html, 'lxml')
    infotable = soup.find_all("table", summary = "Form 13F-NT Header Information")
    rows = infotable[0].find_all('tr')
    for row in rows[3:]:
        cells = row.find_all('td')
        Companies.append(cells[0].getText().encode('utf-8'))
    return Companies

def getData(NBList):
    name_of_issuer = []
    NBList = filter(None, NBList.split('\n'))
    for i in range(2,len(NBList)):
        cells = re.split(r'\s{2,}', NBList[i])
        name_of_issuer.append(cells[0])
    return name_of_issuer
            
def getCompanyNamesTXT(link):
    CompanyNames = []
    html = urllib2.urlopen('https://www.sec.gov'+link).read()
    html = html.upper()
    html = html.replace('<S>', '  ')
    html = html.replace('<C>', '  ')
    index = html.find('FORM 13F INFORMATION')
    html2 = html[index:]
    index = html2.find('NAME OF ISSUER')
    html2 = html2[index:]
    while(html2.find('NAME OF ISSUER') != -1):
        start = html2.find('NAME OF ISSUER')
        remaining = html2[start:]
        end = html2.find('</TABLE>')
        CompanyNames.extend(getData(html2[start:end]))
        html2 = remaining[end:]
    return CompanyNames

def getCompanies(link):
    Companies = []
    if(link[-4:]=='.xml'):
        Companies = getCompanyNames(link)
    if(link[-4:]=='.txt'):
        Companies = getCompanyNamesTXT(link)
    return Companies

In [8]:
import urllib2
import pandas as pd


CIK = []
text_link = []
text_Name = []
text_date = []

input_file = open('C:\Users\Nick\Documents\Current Courses\Web Data Analytics\WDA-Assignment 1\Assignment_1\Assignment_1/HW_Mutual_Fund_Info_Table_Link.csv', 'r')

rows = input_file.readlines()
input_file.close()

# Your Code on Enumerating the Lists.  The result should be three lists, text_link, text_Name,
#and text_date.  Each should have length 122.

# Since we have a comma separated file, I created a dataframe out of the rows.
df = pd.DataFrame([row.split(",") for row in rows[1:]])

# I used the first row for column headers for the aforementioned dataframe.
df.columns = rows[0].split(", ")

# Since every row has a end of line character as a column, I removed them here
del df['\n']

# Reset the index since the indexes are messed up after xml files are dropped.
df = df.reset_index(drop = True)

df1=df.groupby(['CIK','Name'], as_index=False)['Date'].count()
df1=df1[df1['Date']>=2]

CIKdropList = ['0000711202', '0000003521', '0000002110', '0000777535', '0001029068','0000355437']

df = df[(df['CIK'].isin(df1['CIK'].tolist()))]
df = df[~df['CIK'].isin(CIKdropList)]
df = df.reset_index(drop = True)

df = df.sort_values('Date').groupby('CIK').tail(2)
df = df.reset_index(drop = True)
# Populate the required lists using the respective columns from the data frame.
CIK = df['CIK'].tolist()
xml_link = df['Information Table Links'].tolist()
xml_Name = df['Name'].tolist()
xml_date = df['Date'].tolist()

df = df[['CIK', 'Name', 'Information Table Links', 'File Type']]
set(df['CIK'].tolist())

{'0000772129', '0000813470', '0001000207', '0001311981'}

In [13]:
CompanyList = list(set(df['Name'].tolist()))

for i in CompanyList:
    df_new = df.loc[df['Name']==CompanyList[1]]
    InfoLinks = df_new['Information Table Links'].tolist()
    OldCompanies = getCompanies(InfoLinks[0])
    NewCompanies = getCompanies(InfoLinks[1])
    added = set(NewCompanies) - set(OldCompanies)
    dropped = set(OldCompanies) - set(NewCompanies)

print len(added), len(dropped)

46 41


In [15]:
dropped

{'',
 'AERCAP HOLDINGS NV',
 'AEROPOSTALE',
 'ALBANY MOLECULAR RESH INC',
 'ALCATEL-LUCENT',
 'ALTEVA',
 'ASSURED GUARANTY LTD',
 'BEBE STORES INC',
 'BROADRIDGE FINL SOLUTIONS IN',
 'CAL MAINE FOODS INC',
 'CHRISTOPHER & BANKS CORP',
 'COCA COLA BOTTLING CO CONS',
 'COEUR D ALENE MINES CORP IDA',
 'COMVERSE INC',
 'COSI INC',
 'COWEN GROUP INC NEW',
 'DOLE FOOD CO INC NEW',
 'EMULEX CORP',
 'GENWORTH FINL INC COM',
 'GFI GROUP INC',
 'HARDINGE INC',
 'IAC INTERACTIVECORP',
 'INTRALINKS HLDGS INC',
 'JOURNAL COMMUNICATIONS INC',
 'KEYNOTE SYS INC',
 'KROGER CO',
 'MATRIX SVC CO',
 'MONTPELIER RE HOLDINGS LTD',
 'MYERS INDS INC',
 'NN INC',
 'OMEGA PROTEIN CORP',
 'PNM RES INC',
 'REAL GOODS SOLAR INC',
 'SPRINT NEXTEL CORP',
 'TELLABS INC',
 'TETRA TECHNOLOGIES INC DEL',
 'TRIQUINT SEMICONDUCTOR INC',
 'UNS ENERGY CORP',
 'VASCULAR SOLUTIONS INC',
 'VISHAY INTERTECHNOLOGY INC COM COM',
 'YAHOO INC'}

In [ ]:
resultFyle = open("1000207add",'w')
OutputFile = open("1000207drop", "w")

In [ ]:
for r in list(added):
    resultFyle.write(r + "\n")
resultFyle.close()
    
for r in list(dropped):
    OutputFile.write(r + "\n")
OutputFile.close()

In [ ]:
# ONLY NEEDED IF YOU ARE SCRAPING THE LINKS OF THE CIK. OUR ASSIGNMENT 1 HAS ALL THE LINKS FROM THE CIKs
CIK = ['0000002110',
       '0000003521',
       '0000355437',
       '0000711202',
       '0000749155',
       '0000772129',
       '0000777535', 
       '0000797136',
       '0000813470',
       '0000917124',
       '0001000207',
       '0001002556',
       '0001029068',
       '0001157756',
       '0001311981', 
       '0001432353']
Urls = []
for i in range(0, len(CIK)):# 
    myopener = MyOpener()
    page=myopener.open('https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK='+CIK[i]+'&type=N-Q&dateb=&count=100&scd=filings')
    
    html = page.read()
    soup = bs.BeautifulSoup(html, 'lxml')
    
    dateText = soup.find_all('td', class_='small')
    for i in range(0,len(dateText)):
        date = dateText[i].find_next_sibling('td').getText()
        date = getDate(date)
        if (date >= datetime.datetime(2016,9,1)):
            DocumentLink = soup.find_all('a', id = 'documentsbutton')
            Urls.append('https://www.sec.gov' + DocumentLink[i]['href'])
Urls

In [ ]:
date=[]
info_table = []
info_type = []

for page in Urls:
    html = urllib2.urlopen(page).read()   
    date_index = html.find('div class=\"infoHead\">Filing Date<')
    date_html = html[date_index:]
    date_index2 = date_html.find('class=\"info\">')
    date_html = date_html[date_index2:]
    start = date_html.find(">")
    end = date_html.find("<")
    date.append(date_html[start+1:end])  # This gets us the date    
    if html.find('td scope=\"row\">2<') != -1: # By this condition we check whether there is a xml version. For an xml version there are two items present in each table in the page        
        index = html.find('td scope=\"row\">2<')
        html2 = html[index:]    
        start = html2.find('a href=\"/Archives/edgar/data/')
        remaining = html2[start:]
        end = remaining.find('>')    
        info_table.append(remaining[8:end-1]) # we collect the link    
        info_type.append('xml') # we mention that this is an xml link            
    else: # Or else, we get the idea that the info table is in text format        
        index = html.find('td scope=\"row\">Complete submission text file')
        html2 = html[index:]        
        start = html2.find('a href=\"/Archives/edgar/data/')
        remaining = html2[start:]
        end = remaining.find('>')    
        info_table.append(remaining[8:end-1]) 
        info_type.append('text') 